In [1]:
import os
os.chdir("../../scVI/")
os.getcwd()

'/data/yosef2/users/chenling/scVI'

In [2]:
import torch
import pickle
import seaborn as sns
import numpy as np
import pandas as pd
from umap import UMAP
from sklearn.cluster import SpectralClustering
from scvi.inference import UnsupervisedTrainer
from scvi.models import VAE
save_path = '../CSF/Notebooks/'

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline

In [3]:
from random import sample
from numpy.random import permutation


In [4]:
%matplotlib inline

In [5]:
celllabels = np.load(save_path + 'meta/celllabels.npy')
isCD4 = celllabels=='CD4'

In [6]:
clusters = np.load(save_path + 'meta/CD4.clusters.npy')

In [7]:
isMS = np.load(save_path+'meta/isMS.npy')[isCD4]
isCSF = np.load(save_path+'meta/isCSF.npy')[isCD4]

In [8]:
def ES_fast(score, s , p, interval):
    N = len(s)
    N_H = np.sum(s==1)
    m = 1/(N-N_H)
    power = np.abs(score)**p 
    N_R = np.sum(power[s==1])
    h = power / N_R
    ES = [0]
    hit = 0
    miss = 0
    for i in np.arange(0, (len(power)-interval),interval):
        x = np.arange(i,i+interval,1)
        si = s[x]
        hit = hit + np.sum(h[x][si==1])
        miss = miss + m*np.sum(si==0)
        ES.append(hit-miss)
    return(ES)

In [122]:
with open('../CSF/signatures/allsigs.txt') as f:
    geneset = f.readlines()

setname=[]
setorig=[]
genes = []
for x in geneset:
    x = x.rstrip()
    line = x.split('\t')
    if len(line)>1:
        setname.append(line[0])
        setorig.append(line[1])
        genes.append(line[2:])
    

In [125]:
len(np.unique(genes))

101

In [111]:
VisionScore = pd.read_csv('../CSF/signatures/sigScore.allsigs.csv')
VisionScore = VisionScore[VisionScore.columns[1:]]

In [112]:
VisionScore.shape

(25105, 64)

In [10]:
from copy import deepcopy

In [11]:
res = []
for tissue in [True,False]:
    ESscore = []
    for x in VisionScore.columns:
        print(x)
        s = isMS[isCSF==tissue]
        score = np.asarray(VisionScore[x])[isCSF==tissue]
        ranked = pd.DataFrame(np.asarray([s, score]).T,columns=['s','score'])
        ranked = ranked.sample(frac=1)
        ranked = ranked.sort_values(by='score',ascending=False)
        s = np.asarray(ranked['s'])
        score = np.asarray(ranked['score'])
        ES = ES_fast(score,s,1,1)
#         ESscore.append(ES)
        ES_null = []
        for i in range(100):
            s_rand = deepcopy(s)
            np.random.shuffle(s_rand)
            ES_null.append(np.max(ES_fast(score,s_rand,1,1)))
        ESscore.append(np.mean(np.max(ES) > np.asarray(ES_null)))
    res.append(ESscore)
        

G1/S
G2/M
Temra
Th2
Th17
TFH
Tregs
gdT
microglia
Patho-manual
CD4_memory_vs_Naive
TFH_core
TFH_expanded
G1/S_Phase
G2/M_Phase
M_Phase
M/G1_Phase
S_Phase
codeset
Alpha6-Beta4-Integrin_EXP
Androgen-Receptor_EXP
B-Cell-Receptor_EXP
EGFR1_EXP
Hedgehog_EXP
ID_EXP
IL-1_EXP
IL-2_EXP
IL-3_EXP
IL-4_EXP
IL-5_EXP
IL-6_EXP
IL-7_EXP
IL-9_EXP
Kit-Receptor_EXP
Notch_EXP
T-Cell-Receptor_EXP
TGF-beta-Receptor_EXP
TNF-alpha_EXP
Wnt_EXP
ITREG_vs_other_Thelper
NTREG_vs_other_Thelper
Naive_vs_other_Thelper
TH17_vs_other_Thelper
TH1_vs_other_Thelper
TH2_vs_other_Thelper
STAT3 KO vs. WT (Th17)
STAT5 KO vs. WT (Th17)
RORC KO vs. WT (Th17)
EGR2
ETV6
FAS
IKZF4
IL23+TGFB+IL6 vs TGFB+IL6 (Th17)
IRF8
MINA
PLAC8
Th1
Memory
Patho-large
Pathogenic vs. non pathogenic Th17 cells
Th0 NaCl vs. Th0
CD4_exausted_vs_Naive
CD8_exausted
CD8_memory
G1/S
G2/M
Temra
Th2
Th17
TFH
Tregs
gdT
microglia
Patho-manual
CD4_memory_vs_Naive
TFH_core
TFH_expanded
G1/S_Phase
G2/M_Phase
M_Phase
M/G1_Phase
S_Phase
codeset
Alpha6-Beta4-Integri

In [33]:
ES_max = []
Leading_edge = []
for tissue in [True,False]:
    es_max = []
    edge=[]
    for x in VisionScore.columns:
        s = isMS[isCSF==tissue]
        score = np.asarray(VisionScore[x])[isCSF==tissue]
        ranked = pd.DataFrame(np.asarray([s, score]).T,columns=['s','score'])
        ranked = ranked.sample(frac=1)
        ranked = ranked.sort_values(by='score',ascending=False)
        s = np.asarray(ranked['s'])
        score = np.asarray(ranked['score'])
        ES = ES_fast(score,s,1,1)
        es_max.append(np.max(ES))
        edge.append(np.argmax(ES))
    ES_max.append(es_max)
    Leading_edge.append(edge)
        

In [22]:
np.asarray(VisionScore.columns)[(1-np.asarray(res[1])<0.01) & (1-np.asarray(res[0])<0.01)]

array(['Temra', 'Th2', 'gdT', 'microglia', 'Patho-manual', 'TFH_core',
       'M_Phase', 'M/G1_Phase', 'codeset', 'Alpha6-Beta4-Integrin_EXP',
       'Androgen-Receptor_EXP', 'ID_EXP', 'Notch_EXP',
       'TGF-beta-Receptor_EXP', 'Wnt_EXP', 'NTREG_vs_other_Thelper',
       'Naive_vs_other_Thelper', 'TH1_vs_other_Thelper',
       'STAT3 KO vs. WT (Th17)', 'RORC KO vs. WT (Th17)', 'EGR2', 'FAS',
       'Th1', 'Patho-large', 'Pathogenic vs. non pathogenic Th17 cells',
       'Th0 NaCl vs. Th0'], dtype=object)

In [23]:
import pickle
with open(save_path + 'CSEA/allsigs.pkl', 'wb') as f:
    pickle.dump(res, f)


In [120]:
VisionScore_rand = pd.read_csv('../CSF/signatures/sigScore.randsigs.rep100.csv')
VisionScore_rand = VisionScore_rand[VisionScore_rand.columns[1:]]

In [121]:
VisionScore_rand.shape

(25105, 9600)

In [101]:
ES_max_rand = []
Leading_edge_rand = []
for tissue in [True,False]:
    res = []
    edge=[]
    for x in VisionScore_rand.columns:
        s = isMS[isCSF==tissue]
        score = np.asarray(VisionScore_rand[x])[isCSF==tissue]
        ranked = pd.DataFrame(np.asarray([s, score]).T,columns=['s','score'])
        ranked = ranked.sample(frac=1)
        ranked = ranked.sort_values(by='score',ascending=False)
        s = np.asarray(ranked['s'])
        score = np.asarray(ranked['score'])
        ES = ES_fast(score,s,1,1)
        res.append(np.max(ES))
        edge.append(np.argmax(ES))
    ES_max_rand.append(res)
    Leading_edge_rand.append(edge)
        

In [102]:
len(ES_max_rand)

2

In [28]:
# res_control = []
# for tissue in [True,False]:
#     ESscore = []
#     for x in VisionScore_rand.columns:
#         print(x)
#         s = isMS[isCSF==tissue]
#         score = np.asarray(VisionScore_rand[x])[isCSF==tissue]
#         ranked = pd.DataFrame(np.asarray([s, score]).T,columns=['s','score'])
#         ranked = ranked.sample(frac=1)
#         ranked = ranked.sort_values(by='score',ascending=False)
#         s = np.asarray(ranked['s'])
#         score = np.asarray(ranked['score'])
#         ES = ES_fast(score,s,1,1)
# #         ESscore.append(ES)
#         ES_null = []
#         for i in range(100):
#             s_rand = deepcopy(s)
#             np.random.shuffle(s_rand)
#             ES_null.append(np.max(ES_fast(score,s_rand,1,1)))
#         ESscore.append(np.mean(np.max(ES) > np.asarray(ES_null)))
#     res_control.append(ESscore)
        

random0
random1
random2
random3
random4
random5
random6
random7
random8
random9
random10
random11
random12
random13
random14
random15
random16
random17
random18
random19
random20
random21
random22
random23
random24
random25
random26
random28
random29
random30
random31
random32
random33
random34
random35
random36
random37
random38
random39
random40
random41
random42
random43
random44
random46
random47
random48
random49
random50
random51
random52
random53
random54
random55
random56
random57
random58
random59
random60
random61
random62
random63
random64
random65
random66
random67
random68
random69
random70
random71
random72
random73
random74
random75
random76
random77
random78
random79
random80
random81
random82
random83
random85
random86
random87
random88
random89
random90
random91
random92
random93
random94
random95
random96
random97
random98
random99
random100
random101
random102
random103
random104
random105
random106
random107
random0
random1
random2
random3
random4
random5
random6
r

In [34]:
from statsmodels.stats.multitest import multipletests

In [39]:
res = [np.asarray(x) for x in res]
res_control = [np.asarray(x) for x in res_control]

In [73]:
res_control = [x[:64] for x in res_control]

In [74]:
multipletests(1-res[0],method='fdr_bh')[1]

array([0.        , 0.46666667, 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.92734694, 0.        , 0.        , 0.05688889, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 1.        , 0.        , 0.        ,
       1.        , 1.        , 1.        , 0.04363636, 1.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.29957447, 0.        , 0.01488372,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 1.        , 1.        , 0.2226087 ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [48]:
1-res[0]

array([0.  , 0.35, 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71,
       0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       1.  , 0.  , 0.  , 0.98, 1.  , 1.  , 0.03, 1.  , 1.  , 0.  , 1.  ,
       0.  , 0.  , 1.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.  ,
       0.01, 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.96, 0.98, 0.16,
       0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [75]:
temp = pd.DataFrame([1-res[0],multipletests(1-res[0],method='fdr_bh')[1],
              1-res_control[0],multipletests(1-res_control[0],method='fdr_bh')[1],
              ES_max[0],Leading_edge[0],
              1-res[1],multipletests(1-res[1],method='fdr_bh')[1],
              1-res_control[1],multipletests(1-res_control[1],method='fdr_bh')[1],
              ES_max[1],Leading_edge[1]], 
              index=['CSF Pvalue','CSF corrected Pvalue','CSF Pvalue control','CSF Pvalue control corrected',
                    'CSF ES_max','CSF leading edge', 
                    'blood Pvalue','blood corrected Pvalue','blood Pvalue control','blood Pvalue control corrected',
                    'blood ES_max','blood leading edge', ])

In [128]:
temp

,CSF Pvalue,CSF corrected Pvalue,CSF Pvalue control,CSF Pvalue control corrected,CSF ES_max,CSF leading edge,blood Pvalue,blood corrected Pvalue,blood Pvalue control,blood Pvalue control corrected,blood ES_max,blood leading edge
G1/S,0.00,0.000000,0.00,0.000000,0.003443,88.0,0.12,0.219429,0.00,0.000000,0.001604,34.0
G2/M,0.35,0.466667,1.00,1.000000,0.000487,11.0,0.45,0.702439,1.00,1.000000,0.000278,12.0
Temra,0.00,0.000000,0.00,0.000000,0.243513,1943.0,0.00,0.000000,1.00,1.000000,0.085556,386.0
Th2,0.00,0.000000,0.00,0.000000,0.444984,2563.0,0.00,0.000000,0.00,0.000000,0.224561,2209.0
Th17,1.00,1.000000,1.00,1.000000,0.055280,515.0,1.00,1.000000,1.00,1.000000,0.003967,80.0
TFH,0.00,0.000000,0.56,0.746667,0.053675,587.0,0.04,0.075294,0.99,1.000000,0.006355,135.0
Tregs,0.00,0.000000,1.00,1.000000,0.172921,1343.0,1.00,1.000000,0.99,1.000000,0.091453,2017.0
gdT,0.00,0.000000,1.00,1.000000,0.443760,4752.0,0.00,0.000000,0.00,0.000000,0.353220,6884.0
microglia,0.00,0.000000,0.00,0.000000,0.367123,2928.0,0.00,0.000000,0.00,0.000000,0.385070,4953.0
Patho-manual,0.00,0.000000,0.00,0.000000,0.402739,3908.0,0.00,0.000000,0.00,0.000000,0.394049,5433.0


In [76]:
temp = temp.T

In [77]:
temp.index = list(VisionScore.columns)

In [82]:
temp.loc[(temp['CSF Pvalue']<0.05) & (temp['CSF Pvalue control']>0.05) & 
         (temp['blood Pvalue']<0.05) & (temp['blood Pvalue control']>0.05) & 
        (temp['CSF leading edge']<1000) & (temp['blood leading edge']<1000) ]

,CSF Pvalue,CSF corrected Pvalue,CSF Pvalue control,CSF Pvalue control corrected,CSF ES_max,CSF leading edge,blood Pvalue,blood corrected Pvalue,blood Pvalue control,blood Pvalue control corrected,blood ES_max,blood leading edge
TFH,0.0,0.0,0.56,0.746667,0.053675,587.0,0.04,0.075294,0.99,1.0,0.006355,135.0
Th1,0.0,0.0,1.00,1.000000,0.076904,594.0,0.00,0.000000,1.00,1.0,0.029028,397.0


In [83]:
temp.to_csv(save_path+'CSEA/supptab.csv')